In [1]:
import glob
import redis
import configparser
from hashlib import sha256
import json
import yaml
from pprint import pprint

In [2]:
metadata_files = []
for filename in glob.iglob('../' + '**/**model-metadata.yaml', recursive=True):
     metadata_files.append(filename)

In [3]:
print(*metadata_files, sep = "\n")

../Atlas-Integration/consumption-model-metadata.yaml
../Atlas-Integration/asset-wealth-model-metadata.yaml
../DSSAT-Integration/DSSAT-model-metadata.yaml
../Kimetrica-Integration/population-model-metadata.yaml
../Kimetrica-Integration/malnutrition-model-metadata.yaml
../CHIRPS-Integration/CHIRPS-model-metadata.yaml
../FSC-Integration/FSC-model-metadata.yaml


In [4]:
config = configparser.ConfigParser()
config.read('../REST-Server/config.ini')

r = redis.Redis(host=config['REDIS']['HOST'],
                port=config['REDIS']['PORT'],
                db=config['REDIS']['DB'])

In [5]:
models = {}

for m in metadata_files:
    with open(m, 'r') as stream:
        model = yaml.safe_load(stream)
        models[model['id']] = model

In [6]:
concepts = {}

# for each model
for kk, vv in models.items():
    
    # get its concepts
    for cc in vv.get('concepts',[]):
        
        # if concept not in concepts dict, add it
        if cc not in concepts:
            concepts[cc] = set()
            concepts[cc].add(kk)
            
        # if concept in concepts, add model to set
        else:
            concepts[cc].add(kk)

In [7]:
print("Concept mapping to models:")
pprint(concepts)

Concept mapping to models:
{'GPE': {'population_model'},
 'assets': {'consumption_model', 'asset_wealth_model'},
 'climate': {'CHIRPS'},
 'crop_storage': {'FSC', 'DSSAT'},
 'drought': {'CHIRPS'},
 'economy': {'consumption_model', 'FSC', 'asset_wealth_model'},
 'export': {'FSC'},
 'farming': {'DSSAT'},
 'flooding': {'CHIRPS'},
 'food_insecurity': {'malnutrition_model'},
 'food_production': {'DSSAT'},
 'food_security': {'malnutrition_model'},
 'import': {'FSC'},
 'inflation': {'consumption_model'},
 'material': {'asset_wealth_model'},
 'nutrient': {'malnutrition_model'},
 'population': {'population_model'},
 'poverty': {'consumption_model', 'asset_wealth_model'},
 'precipitation': {'CHIRPS'},
 'water': {'CHIRPS'}}


In [8]:
concept_names = set(concepts.keys())

In [9]:
# add concepts to "concepts" set in Redis
for c in concept_names:
    r.sadd('concepts', c)

In [10]:
print("We can obtain all concepts:")
pprint(r.smembers('concepts'))

We can obtain all concepts:
{b'GPE',
 b'assets',
 b'climate',
 b'crop_storage',
 b'drought',
 b'economy',
 b'export',
 b'farming',
 b'flooding',
 b'food_insecurity',
 b'food_production',
 b'food_security',
 b'import',
 b'inflation',
 b'material',
 b'nutrient',
 b'population',
 b'poverty',
 b'precipitation',
 b'water'}


In [11]:
#for each concept
for cc, vv in concepts.items():
    
    # for each model associated with each concept
    for ee in vv:
        
        # add the model to a Redis set named for the concept name
        r.sadd(cc, ee)

In [12]:
print("We can obtain the models associated with 'economy', for example:")
pprint(r.smembers('economy'))

We can obtain the models associated with 'economy', for example:
{b'consumption_model', b'FSC', b'asset_wealth_model'}
